# Prepare the database for the REST Service Examples

This notebook prepares the database for creating and running the rest services shown in the Db2 RESTful External Py UDF.ipynb notebook.  It does things like creating roles and granting permissions.  It uses the Db2 Magic commands to interact with the database.  You can see how to install and use Db2 Magic at this repo:  https://github.com/IBM/db2-jupyter

#####  If you are looking at the REST example, make sure to start by examining the Db2 RESTful External Py UDF.ipynb notebook.

Conversly, you can execute the SQL/DDL shown here in your favorite Db2 client if this all seems like too much trouble.  

## Run the notebook to set up Db2 Magic

In [ ]:
!wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb

In [1]:
# This is the newest notebook and has graphing capabilities
%run "db2 (4).ipynb"

         Install PANDAS version 1.3+ for more efficient dataframe creation.
         parallelism to work.
         Install MULTIPROCESSING/MULTIPROCESS(OSX) if you want allow
         multiple SQL threads to run in parallel.
Db2 Extensions Loaded.


### Connect to the Database

Connect with a user with DBADM and SECADM authorities. Change these values to connect to your database.

In [6]:
DB="SAMPLE"
USER="db2inst1"
PW="ibmdb2aa"
HOST="localhost"
PORT=50000

In [8]:
%sql CONNECT TO {DB} USER {USER} USING {PW} HOST {HOST} PORT {PORT}

Connection successful.


##### Create the needed schemas

EXTPY schema holds objects for the external Python UDF

REST_SERVICES schema holds objects used by the Db2 REST Endpoint

In [ ]:
%%sql
CREATE SCHEMA EXTPY;
CREATE SCHEMA REST_SERVICES;

##### Create the needed roles

In [4]:
%%sql
drop role service_admin;
drop role service_user;
create role service_admin;
create role service_user;

Command completed.


##### Grant the roles to users

In [5]:
%%sql
GRANT ROLE service_admin TO USER service_admin1;
GRANT ROLE service_user TO USER service_user1;

Command completed.


### The SERVICE_ADMIN role is intended to users who create services

- Min privileges to create, execute and describe services
- Also allows GRANT and REVOKE permissions to service users
- Allows token creation for this user
- Allows all tables in the GOSALES schema to be selected.
- Assumes REST_SERVICES schema already created


In [6]:
%%sql 
GRANT CONNECT ON DATABASE TO ROLE SERVICE_ADMIN;
GRANT SELECTIN ON SCHEMA SYSCAT TO ROLE SERVICE_ADMIN;
GRANT SELECT, INSERT, UPDATE, DELETE ON DB2REST.RESTSERVICE TO ROLE SERVICE_ADMIN;
GRANT SELECT ON SYSIBM.SYSDUMMY1 TO ROLE SERVICE_ADMIN;
GRANT ALL ON SCHEMA REST_SERVICES TO ROLE SERVICE_ADMIN WITH GRANT OPTION;
GRANT EXECUTE ON PROCEDURE SYSPROC.ADMIN_CMD TO ROLE SERVICE_ADMIN;
GRANT EXECUTE ON FUNCTION SYSPROC.AUTH_LIST_AUTHORITIES_FOR_AUTHID TO ROLE SERVICE_ADMIN;

-- Grant authorities on tables used in any service created by this user id
GRANT SELECTIN, INSERTIN, UPDATEIN, DELETEIN, EXECUTEIN ON SCHEMA EXTPY TO ROLE SERVICE_ADMIN;

Command completed.


### The SERVICE_USER role is intended for users who execute the services

- Mostly just needs authority to execue services created by others


In [7]:
%%sql
GRANT CONNECT ON DATABASE TO ROLE SERVICE_USER;
GRANT EXECUTE ON PROCEDURE SYSPROC.ADMIN_CMD TO ROLE SERVICE_USER;
GRANT EXECUTEIN ON SCHEMA REST_SERVICES TO ROLE SERVICE_USER;
GRANT EXECUTE ON FUNCTION SYSPROC.AUTH_LIST_AUTHORITIES_FOR_AUTHID TO ROLE SERVICE_USER;

Command completed.


##### Create the table used by the UDF when executed with table data.  This is described in section 4 of the readme

In [ ]:
%%sql
drop table if exists extpy.person_features;
CREATE TABLE extpy.person_features (
ID INT NOT NULL GENERATED ALWAYS AS IDENTITY (START WITH 1, INCREMENT BY 1, NO CACHE),
FEMALE FLOAT,
MALE FLOAT,
MARRIED FLOAT,
SINGLE FLOAT,
UNSPECIFIED FLOAT,
EXECUTIVE FLOAT,
HOSPITALITY FLOAT,
OTHER FLOAT,
PROFESSIONAL FLOAT,
RETAIL FLOAT,
RETIRED FLOAT,
SALES FLOAT,
STUDENT FLOAT,
TRADES FLOAT,
CAMPING_EQUIPMENT FLOAT,
GOLF_EQUIPMENT FLOAT,
MOUNTAINEERING_EQUIPMENT FLOAT,
OUTDOOR_PROTECTION FLOAT,
PERSONAL_ACCESSORIES FLOAT,
AGE FLOAT,
IS_TENT  FLOAT,
PRIMARY KEY (ID))
ORGANIZE BY ROW;

In [5]:
%sql CONNECT RESET

Connection closed.
